# grocery_sales_forecasting_NonParametric

This baseline model is non-parametric. This model is simply average of history sale (from 2013 to 2016 of the same day.)

In [1]:
# Import libraries necessary for this project
import os.path
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from IPython.display import display
import matplotlib.pyplot as plt

%matplotlib inline

def loaddata(filename, nrows=None):
    types = {'id': 'int32', 'item_nbr': 'int32', 'store_nbr': 'int16', 'unit_sales': 'float32', 'onpromotion': bool,}
    data = pd.read_csv(filename, parse_dates=['date'], dtype=types, nrows=nrows, infer_datetime_format=True)
    return data

In [ ]:
test_data = loaddata("input/test.csv")
print("Favorita grocery sales forecasting testing data has {} samples with {} features each.".format(*test_data.shape))

years = [2013, 2014, 2015, 2016]
for year in years:
    pickle_file = 'train_' + str(year) + '_8.pkl'
    train_data = pickle.load(open( pickle_file, "rb" ))
    for item in range(test_data.shape[0]):
        date_test = test_data.at[item, 'date']
        test_store_nbr = test_data.at[item,'store_nbr']
        test_item_nbr = test_data.at[item,'item_nbr']

        date_hist = date_test.replace(year = year)
        mask = (train_data['date']==date_hist)&(train_data['store_nbr'] == test_store_nbr)&(train_data['item_nbr'] == test_item_nbr)
        result = train_data.loc[mask, 'unit_sales']
        if result.shape[0]>0:
            test_data.at[item, str(year)] = result.iloc[0]
    print(year, 'is completed!')

test_data.to_csv('test_data.csv', sep='\t')


In [26]:
test_data = pd.read_csv('test_data.csv', sep='\t')

#test_data['unit_sales'] = test_data[['2013', '2014', '2015', '2016']].mean(axis=1)
test_data['unit_sales'] = test_data[['2013', '2014', '2015', '2016']].median(axis=1)
submission = test_data[['id', 'unit_sales']].set_index('id').fillna(0)
result['unit_sales'] = result['unit_sales'].apply(lambda x: x if x >=0 else 0)
result.to_csv('submission.csv')
display(result.head(6))

,Unnamed: 0,id,date,store_nbr,item_nbr,onpromotion,2013,2014,2015,2016
0,0,125497040,2017-08-16,1,96995,False,3.0,NaN,NaN,NaN
1,1,125497041,2017-08-16,1,99197,False,NaN,NaN,NaN,NaN
2,2,125497042,2017-08-16,1,103501,False,NaN,NaN,NaN,NaN
3,3,125497043,2017-08-16,1,103520,False,NaN,3.0,NaN,NaN
4,4,125497044,2017-08-16,1,103665,False,2.0,NaN,1.0,NaN
5,5,125497045,2017-08-16,1,105574,False,4.0,6.0,3.0,3.0


,unit_sales
id,
125497040,3.0
125497041,0.0
125497042,0.0
125497043,3.0
125497044,1.5
125497045,4.0


In [ ]:
#Load one year's train data;  
train_pkl = 'train_2017.pkl'
years = ['2013', '2014', '2015', '2016', '2017']
train_data = pickle.load(open( train_pkl, "rb" ))

#Add oil price of corresponding date; 
oil = pd.read_csv("input/oil.csv")
oil['date'] = pd.to_datetime(oil['date'], format='%Y-%m-%d')
train_data= pd.merge(train_data,oil, right_on='date',left_on='date',how='left')

#Add transaction of coresponding date and store;
transactions = pd.read_csv("input/transactions.csv")
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')
train_data= pd.merge(train_data,transactions, left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr'], how='left')
display(train_data.tail(5))
display(train_data.describe())

In [ ]:
# Separate training feature and training target
target_raw = train_data['unit_sales']
features_raw = train_data.drop(['id', 'unit_sales'], axis=1)

#Drop items with missing data
features_raw.replace(["NaN", 'NaT'], np.nan, inplace = True)
features_raw.dropna(inplace=True)
# features_clean_NaN = features_raw[~features_raw.isin(['NaN', 'NaT']).any(axis=1)] #computationally expensive

#Categorical feature
features_raw['onpromotion'] = features_raw['onpromotion'].apply(lambda x: 1 if x else 0)

#Numerical feature
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['dcoilwtico', 'transactions']
features_minmax_transform = pd.DataFrame(data = features_raw)
features_minmax_transform[numerical] = scaler.fit_transform(features_minmax_transform[numerical])
display(features_minmax_transform.tail(5))
display(features_minmax_transform.describe())

Naive Model: Search the history record, and average it